# Example of DOV search methods for quartaire stratigrafie

@pjhaest
date 20190215

## Use cases:
* Select records in a bbox
* Select records in a bbox with selected properties
* Select records within a distance from a point
* Select records in a municipality
* Get records using info from wfs fields, not available in the standard output dataframe

In [9]:
%matplotlib inline
import os, sys
import inspect
import warnings; warnings.simplefilter('ignore')

In [10]:
# check pydov path
import pydov

## Get information about code base

In [11]:
from pydov.search.interpretaties import QuartairStratigrafieSearch
# information about the QuartairStratigrafie type (In Dutch):
ip_quart = QuartairStratigrafieSearch()
print(ip_quart.get_description())

De quartaire afzettingen in Vlaanderen, de jongste afzettingen, zijn lateraal en verticaal heterogeen en varieren erg in dikte. De afzettingen zijn vooral onder continentale omstandigheden tot stand gekomen (rivier-, duinafzettingen). Eigenlijk zijn het pakketten die zelden meer dan 30 meter en soms zelfs minder dan 1 meter dik zijn. Omwille van deze grote eigenheid en de grote mate van detail van de beschrijvingen, werd hiervoor een eigen stratigrafische codering uitgewerkt voor Vlaanderen.


De quartaire afzettingen in Vlaanderen, de jongste afzettingen, zijn lateraal en verticaal heterogeen en varieren erg in dikte. De afzettingen zijn vooral onder continentale omstandigheden tot stand gekomen (rivier-, duinafzettingen). Eigenlijk zijn het pakketten die zelden meer dan 30 meter en soms zelfs minder dan 1 meter dik zijn. Omwille van deze grote eigenheid en de grote mate van detail van de beschrijvingen, werd hiervoor een eigen stratigrafische codering uitgewerkt voor Vlaanderen.


In [ ]:
# information about the available fields for a QuartairStratigrafie object
fields = ip_quart.get_fields()
# print available fields
for f in fields.values():
    print(f['name'])

In [14]:
# print information for a certain field
fields['Type_proef']

{'cost': 1,
 'definition': "Het type van de proef waarvan de quartaire stratigrafie een interpretatie is. Bij een quartaire stratigrafie zal dit altijd 'Boring' zijn.",
 'name': 'Type_proef',
 'notnull': False,
 'query': True,
 'type': 'string',
 'values': ['Boring']}

The cost is an arbitrary attribute to indicate if the information is retrieved from a wfs query (cost = 1), 
or from an xml (cost = 10)

## Try-out of use cases

### Select interpretations in a bbox

In [15]:
from pydov.util.location import Within, Box

# Get all interpretations in a bounding box (llx, lly, ulx, uly)
# the pkey_boring link is not available below, but is in the df
df = ip_quart.search(location=Within(Box(132815, 191700, 133815, 192700)))
print(df.head())

[000/003] ccc
                                  pkey_interpretatie  \
0  https://www.dov.vlaanderen.be/data/interpretat...   
1  https://www.dov.vlaanderen.be/data/interpretat...   
2  https://www.dov.vlaanderen.be/data/interpretat...   
3  https://www.dov.vlaanderen.be/data/interpretat...   
4  https://www.dov.vlaanderen.be/data/interpretat...   

                                         pkey_boring  \
0  https://www.dov.vlaanderen.be/data/boring/1974...   
1  https://www.dov.vlaanderen.be/data/boring/1974...   
2  https://www.dov.vlaanderen.be/data/boring/1974...   
3  https://www.dov.vlaanderen.be/data/boring/1974...   
4  https://www.dov.vlaanderen.be/data/boring/1974...   

  betrouwbaarheid_interpretatie         x         y  diepte_laag_van  \
0                      onbekend  132975.0  191969.0              0.0   
1                      onbekend  132975.0  191969.0              4.5   
2                      onbekend  132975.0  191969.0              7.5   
3                      o

### Select interpretations in a bbox with selected properties

In [16]:
# list available query methods
methods = [i for i,j in inspect.getmembers(sys.modules['owslib.fes'], 
                                           inspect.isclass) 
           if 'Property' in i]
print(methods)
from owslib.fes import PropertyIsGreaterThanOrEqualTo

['PropertyIsBetween', 'PropertyIsEqualTo', 'PropertyIsGreaterThan', 'PropertyIsGreaterThanOrEqualTo', 'PropertyIsLessThan', 'PropertyIsLessThanOrEqualTo', 'PropertyIsLike', 'PropertyIsNotEqualTo', 'PropertyIsNull', 'SortProperty']


The property feature methodes listed above are available from the owslib module. These were not adapted for use in pydov.

In [17]:
# Get deep boreholes in a bounding box 
from owslib.fes import PropertyIsLike
# the propertyname can be any of the fields of the interpretations object that belong to the wfs source
# the literal is always a string, no matter what its definition is in the boring object (string, float...)
query = PropertyIsLike(
        propertyname='betrouwbaarheid_interpretatie', literal='onbekend')
df = ip_quart.search(location=Within(Box(132815, 191700, 133815, 192700)),
                     query=query
                    )
print(df.head())

[000/003] ccc
                                  pkey_interpretatie  \
0  https://www.dov.vlaanderen.be/data/interpretat...   
1  https://www.dov.vlaanderen.be/data/interpretat...   
2  https://www.dov.vlaanderen.be/data/interpretat...   
3  https://www.dov.vlaanderen.be/data/interpretat...   
4  https://www.dov.vlaanderen.be/data/interpretat...   

                                         pkey_boring  \
0  https://www.dov.vlaanderen.be/data/boring/1974...   
1  https://www.dov.vlaanderen.be/data/boring/1974...   
2  https://www.dov.vlaanderen.be/data/boring/1974...   
3  https://www.dov.vlaanderen.be/data/boring/1974...   
4  https://www.dov.vlaanderen.be/data/boring/1974...   

  betrouwbaarheid_interpretatie         x         y  diepte_laag_van  \
0                      onbekend  132975.0  191969.0              0.0   
1                      onbekend  132975.0  191969.0              4.5   
2                      onbekend  132975.0  191969.0              7.5   
3                      o

### Select records within a distance from a point

In [23]:
from pydov.util.location import Point, WithinDistance

# Get all interpretations within a defined distance from a point
location = WithinDistance(location=Point(133000, 192000),
                          distance=1000, distance_unit='meter')
df = ip_quart.search(location=location)
print(df.head())

[000/015] .....c.c......c
                                  pkey_interpretatie  \
0  https://www.dov.vlaanderen.be/data/interpretat...   
1  https://www.dov.vlaanderen.be/data/interpretat...   
2  https://www.dov.vlaanderen.be/data/interpretat...   
3  https://www.dov.vlaanderen.be/data/interpretat...   
4  https://www.dov.vlaanderen.be/data/interpretat...   

                                         pkey_boring  \
0  https://www.dov.vlaanderen.be/data/boring/1943...   
1  https://www.dov.vlaanderen.be/data/boring/1943...   
2  https://www.dov.vlaanderen.be/data/boring/1943...   
3  https://www.dov.vlaanderen.be/data/boring/1943...   
4  https://www.dov.vlaanderen.be/data/boring/1943...   

  betrouwbaarheid_interpretatie         x         y  diepte_laag_van  \
0                      onbekend  132089.0  191777.0             0.00   
1                      onbekend  132089.0  191777.0             1.00   
2                      onbekend  132089.0  191777.0             1.50   
3           

### Select interpretations in a municipality

In [18]:
from owslib.fes import PropertyIsEqualTo
query = PropertyIsEqualTo(propertyname='gemeente',
                          literal='Hamme')
df = ip_quart.search(query=query)
print(df.head())

[000/028] cccccccccccccccccccccccccccc
                                  pkey_interpretatie  \
0  https://www.dov.vlaanderen.be/data/interpretat...   
1  https://www.dov.vlaanderen.be/data/interpretat...   
2  https://www.dov.vlaanderen.be/data/interpretat...   
3  https://www.dov.vlaanderen.be/data/interpretat...   
4  https://www.dov.vlaanderen.be/data/interpretat...   

                                         pkey_boring  \
0  https://www.dov.vlaanderen.be/data/boring/1894...   
1  https://www.dov.vlaanderen.be/data/boring/1894...   
2  https://www.dov.vlaanderen.be/data/boring/1935...   
3  https://www.dov.vlaanderen.be/data/boring/1935...   
4  https://www.dov.vlaanderen.be/data/boring/1935...   

  betrouwbaarheid_interpretatie         x         y  diepte_laag_van  \
0                          goed  128930.0  198740.0              0.0   
1                          goed  128930.0  198740.0             12.2   
2                      onbekend  133638.2  197756.1              0.0   

### Visualize results

Using Folium, we can display the results of our search on a map.

In [19]:
# import the necessary modules (not included in the requirements of pydov!)
import folium
from folium.plugins import MarkerCluster
from pyproj import Proj, transform
# for windows 10 users, I could not get it working with Microsoft Edge
# change the browser to Chrome in the Notebook config file instead

In [20]:
# convert the coordinates to lat/lon for folium
def convert_latlon(x1, y1):
    inProj = Proj(init='epsg:31370')
    outProj = Proj(init='epsg:4326')
    x2,y2 = transform(inProj, outProj, x1, y1)
    return x2, y2
df['lon'], df['lat'] = zip(*map(convert_latlon, df['x'], df['y'])) 
# convert to list
loclist = df[['lat', 'lon']].values.tolist()

In [22]:
# initialize the Folium map on the centre of the selected locations, play with the zoom until ok
fmap = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=12)
marker_cluster = MarkerCluster().add_to(fmap)
for loc in range(0, len(loclist)):
    folium.Marker(loclist[loc], popup=df['lid1'][loc]).add_to(marker_cluster)
fmap